# 109 Python intermediate - context manager
_Kamil Bartocha_

_wersja_ 0.0.1

Context Manager

- mechanizm w Pythonie, który pozwala zarządzać zasobami (takimi jak pliki, połączenia z bazą danych, zamykanie socketów itp.) w sposób automatyczny i efektywny. Dzięki context managerowi możemy zapewnić, że zasoby te są poprawnie otwierane i zamykane, nawet w przypadku wystąpienia błędów.

- Context manager w Pythonie wykorzystuje bloki `with`, które gwarantują, że określone czynności zostaną wykonane przed rozpoczęciem pracy oraz po jej zakończeniu, bez konieczności ręcznego zarządzania zasobami.

- Podstawowy Przykład z with (Zarządzanie plikami)

Najczęstszym przykładem jest użycie context managera przy pracy z plikami. Gdy otwieramy plik przy użyciu `with`, Python automatycznie zadba o to, by plik został zamknięty po zakończeniu pracy.

```python
with open('example.txt', 'w') as file:
    file.write('Hello, World!')

```

W tym przykładzie:

- Plik jest otwierany za pomocą `with open()`, a zarządzanie jego zamknięciem jest automatycznie obsługiwane.
- Nawet jeśli wystąpi błąd wewnątrz bloku `with`, Python automatycznie zamknie plik.

### Dlaczego Warto Używać Context Managera?
**1. Automatyczne zarządzanie zasobami:** Context managerzy automatyzują proces zarządzania zasobami, eliminując ryzyko błędów wynikających z niezamkniętych zasobów.

**2. Bezpieczeństwo:** Nawet w przypadku wystąpienia błędów lub wyjątków, zasoby zostaną zwolnione.

**3. Czytelność:** Bloki with sprawiają, że kod jest bardziej czytelny i łatwiejszy do zrozumienia.

### Zastosowania
Context manager jest szczególnie użyteczny w sytuacjach, gdy zasoby muszą być otwierane i zamykane, takich jak:

- Praca z plikami.

- Zarządzanie połączeniami sieciowymi.

- Otwarcie i zamknięcie połączeń do baz danych.

- Zarządzanie blokadami wielowątkowymi (mutex).

## Tworzenie Własnego Context Managera
Możemy również tworzyć własne context managery, używając metod `__enter__` i `__exit__`:

### Dwa Kluczowe Metody Context Managera
Aby stworzyć własny context manager, możemy zdefiniować klasę, która implementuje dwie specjalne metody:

`__enter__()`: Metoda ta jest wykonywana na początku bloku `with`.
`__exit__()`: Metoda ta jest wykonywana po zakończeniu bloku `with`, niezależnie od tego, czy doszło do wyjątku.

In [2]:
class MyContextManager:
    def __enter__(self):
        print("Wchodzę do bloku 'with'")
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        print("Wychodzę z bloku 'with'")
        return False

with MyContextManager() as manager:
    print("Jestem w bloku 'with'")
    raise Exception
    print("Jestem w bloku'")



Wchodzę do bloku 'with'
Jestem w bloku 'with'
Wychodzę z bloku 'with'


Exception: 

In [7]:
class Name:
    def __enter__(self):
        print("Start'")
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        print("End'")

with Name() as manager:
    print("Jestem w bloku 'with'")

Start'
Jestem w bloku 'with'
End'


W metodzie `__exit__`, parametry `exc_type`, `exc_value` i `traceback` są związane z obsługą wyjątków, które mogą wystąpić wewnątrz bloku `with`. Oto, co oznaczają:

1. `exc_type`
To typ wyjątku, który został zgłoszony. Jest to klasa wyjątku (np. `ValueError`, `TypeError`, itp.).
Jeśli nie zgłoszono żadnego wyjątku, wartość `exc_type` wynosi `None`.

2. `exc_value`
To rzeczywisty obiekt wyjątku, czyli instancja klasy wyjątku, która zawiera dodatkowe informacje o błędzie (np. wiadomość błędu).
Jeśli nie zgłoszono żadnego wyjątku, `exc_value` wynosi `None`.

3. `traceback`
To obiekt śledzenia stosu, który zawiera informacje o tym, gdzie w kodzie wystąpił błąd. Obiekt ten umożliwia analizowanie ścieżki wykonania kodu prowadzącej do wystąpienia wyjątku.
Jeśli nie zgłoszono żadnego wyjątku, `traceback` wynosi `None`.

In [10]:
class MyContextManager:
    def __enter__(self):
        print("Wchodzę do kontekstu")
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type:
            print(f"Wystąpił wyjątek: {exc_type.__name__}")
            print(f"Treść wyjątku: {exc_value}")
            return True  # Powoduje zignorowanie wyjątku
        print("Wychodzę z kontekstu")
        return False  # Pozwala na ponowne zgłoszenie wyjątku

with MyContextManager():
    raise ValueError("Błąd testowy")


Wchodzę do kontekstu
Wystąpił wyjątek: ValueError
Treść wyjątku: Błąd testowy


In [ ]:
import sqlite3

class DatabaseManager:
    def __init__(self, db_name):
        self.db_name = db_name
        self.connection = None
    def __enter__(self):
        self.connection = sqlite3.connect(self.db_name)
        return self.connection

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.connection:
            self.connection.commit()  # Zatwierdzanie zmian
            self.connection.close()
        return False

with DatabaseManager('example.db') as conn:
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS users (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        name TEXT,
                        age INTEGER
                    )''')

    cursor.execute('INSERT INTO users (name, age) VALUES (?, ?)', ("Jan", 30))
    cursor.execute(' INTO users (name, age) VALUES (?, ?)', ("Anna", 25))
    cursor.execute('SELECT * FROM users')
    users = cursor.fetchall()
    print(users)



## Dekorator `@contextmanager`
Python oferuje również uproszczony sposób na tworzenie context managerów za pomocą dekoratora `@contextmanager` z modułu `contextlib`. Pozwala to na pisanie context managerów bez konieczności tworzenia pełnej klasy.

In [3]:
from contextlib import contextmanager

@contextmanager
def my_context():
    print("Rozpoczęcie")
    yield  # Zawiesza wykonanie i przekazuje kontrolę do bloku 'with'
    print("Zakończenie")

with my_context():
    #<before yield>
    print("W środku bloku")

Rozpoczęcie
W środku bloku
Zakończenie


Słowo kluczowe `yield` w kontekście dekoratora `@contextmanager` z modułu `contextlib` ma specjalne znaczenie. Umożliwia ono rozdzielenie działań, które mają zostać wykonane przed oraz po wykonaniu bloku `with`. W `contextmanager` pełni funkcję podobną do podziału między metodami `__enter__` i `__exit__` w tradycyjnym context managerze.

**- Przed `yield`:** Cokolwiek jest zapisane przed instrukcją `yield`, zostanie wykonane przed rozpoczęciem pracy w bloku `with`.

**- Po `yield`:** Cokolwiek jest zapisane po instrukcji `yield`, zostanie wykonane po zakończeniu pracy w bloku `with`, nawet jeśli w jego trakcie wystąpi wyjątek.

### Przykład - własny konteks manager do pliku

In [6]:
from contextlib import contextmanager

@contextmanager
def open_file(file_name):
    file = open(file_name, 'r')
    lines = file.readlines()
    try:
        yield lines  # Przekazanie kontroli do bloku 'with'
    finally:
        file.close()  # Zamykanie pliku po zakończeniu bloku 'with'


with open_file('data.json') as f:
    print(f)


['{\n', '    "name": "John",\n', '    "age": 30,\n', '    "city": "Gdańsk"\n', '}']


### Własna implementacja pozwala na rozszerzenie mechanizmów:


In [24]:
import time
from contextlib import contextmanager

@contextmanager
def open_file(file_name, mode='r'):
    """Context manager do otwierania pliku z dodatkowymi funkcjami."""
    start_time = time.time()  # Mierzymy czas rozpoczęcia
    print(f"Otwieranie pliku: {file_name}")

    try:
        file = open(file_name, mode)
        yield file  # Przekazujemy plik do bloku `with`
    except FileNotFoundError as e:
        print(f"Błąd: Plik {file_name} nie został znaleziony.")
        raise e  # Przekazujemy wyjątek dalej
    except Exception as e:
        print(f"Nieznany błąd: {e}")
        raise e  # Przekazujemy dalej inne nieoczekiwane błędy
    finally:
        file.close()  # Zamykanie pliku
        elapsed_time = time.time() - start_time  # Obliczamy czas trwania operacji
        print(f"Zamknięcie pliku: {file_name}")
        print(f"Czas operacji: {elapsed_time:.4f} sekund \n")


try:
    with open_file('data.json', 'r') as f:
        content = f.read()
        # print(content)
except Exception as e:
    print(f"Wystąpił błąd: {e}")


try:
    with open_file('large_file.txt', 'r') as f:
        content = f.read()
        # print(content)
except Exception as e:
    print(f"Wystąpił błąd: {e}")


try:
    with open_file('example.txt', 'r') as f:
        content = f.read()
        # print(content)
except Exception as e:
    print(f"Wystąpił błąd: {e}")

Otwieranie pliku: data.json
Zamknięcie pliku: data.json
Czas operacji: 0.0008 sekund 

Otwieranie pliku: large_file.txt
Zamknięcie pliku: large_file.txt
Czas operacji: 0.0007 sekund 

Otwieranie pliku: example.txt
Błąd: Plik example.txt nie został znaleziony.
Wystąpił błąd: local variable 'file' referenced before assignment
